In [1]:
# paquetes para importar
import requests
import json
import numpy as np
import datetime
import string
import pandas as pd

### 0) Dataset original

In [3]:
df_resume = pd.read_csv('my_data.csv')